In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import tempfile
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import dataset_metadata
import apache_beam as beam
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from apache_beam.options.pipeline_options import PipelineOptions
import tensorflow_transform.beam.impl as beam_impl
 
from measurements import measure

tf.__version__

'1.13.1'

## Preprocessing measurement data for Machine Learning

A little trick: Take a note of the temporary directory containing the data for re-use in other notebooks.

In [5]:
temp_dir = tempfile.mkdtemp()
with open('temp_dir.txt', 'w') as file:
    file.write(temp_dir)
temp_dir

'C:\\Users\\wgi\\AppData\\Local\\Temp\\tmpj9tjlo0n'

In [6]:
# Signature data: Just the way it'll arrive at prediction time
signature_csv_train = os.path.join(temp_dir, "signature_train.csv")
signature_csv_eval = os.path.join(temp_dir, "signature_eval.csv")
signature_csv_test = os.path.join(temp_dir, "signature_test.csv")

# Training data: maybe scaled or further pre-processed.
training_csv = os.path.join(temp_dir, "training.csv")
eval_csv = os.path.join(temp_dir, "eval.csv")

# TFRecord: Allows for high performance input into computational graphs
training_tfr = os.path.join(temp_dir, "training.tfr")
eval_tfr = os.path.join(temp_dir, "eval.tfr")

print("You can find the files at:")
print(signature_csv_train, training_csv, training_tfr)

You can find the files at:
C:\Users\wgi\AppData\Local\Temp\tmpj9tjlo0n\signature_train.csv C:\Users\wgi\AppData\Local\Temp\tmpj9tjlo0n\training.csv C:\Users\wgi\AppData\Local\Temp\tmpj9tjlo0n\training.tfr


## The orginal data

In [7]:
data = measure(5)
data.to_csv(signature_csv_train, index=None)
data = pd.read_csv(signature_csv_train)
data.head()

,beta1,beta2,hour,humidity,weekday
0,3.626056,4.240515,4,24.640224,6
1,-4.905572,1.125959,8,10.155988,6
2,-3.992142,-0.566658,5,11.319303,4
3,2.002481,-2.075683,13,24.520133,5
4,-0.239935,-3.456368,15,20.436716,6


### Specify the input and output formats

In [8]:
ORDERED_SIGNATURE_COLUMNS=["beta1", "beta2", "hour", "humidity", "weekday"]
header = bytes(",".join(ORDERED_SIGNATURE_COLUMNS), 'UTF-8')

In [9]:
feature_spec = {
    'beta1': tf.io.FixedLenFeature([1], tf.float32),
    'beta2': tf.io.FixedLenFeature([1], tf.float32),
    'weekday': tf.io.FixedLenFeature([1], tf.int64),
    'hour': tf.io.FixedLenFeature([1], tf.int64),
    'humidity': tf.io.FixedLenFeature([1], tf.float32)
}
schema = dataset_schema.from_feature_spec(feature_spec)

### Create an encoder and test it

In [10]:
csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)
records = csv_encoder.decode("10.201, 10.101, 3,1.234,4")
print(records)
csv_encoder.encode(records)

{'beta1': array([10.201], dtype=float32), 'beta2': array([10.101], dtype=float32), 'hour': array([3], dtype=int64), 'humidity': array([1.234], dtype=float32), 'weekday': array([4], dtype=int64)}


b'10.201,10.101,3,1.234,4'

---
# The Apache Beam pipeline 

In [11]:
def process_data(row):
    print(row)
    return row

### Dry run - Everything working?

In [12]:
ORDERED_SIGNATURE_COLUMNS

['beta1', 'beta2', 'hour', 'humidity', 'weekday']

In [13]:
with beam.Pipeline('DirectRunner', PipelineOptions()) as p:

    csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)    

    _ = (p 
         | 'read_from_csv' >> beam.io.ReadFromText(
             file_pattern=signature_csv_train, coder=csv_encoder, skip_header_lines=1)
         
         | 'process_records' >> beam.Map(process_data)
         
         | 'write_to_csv' >> beam.io.WriteToText(
             file_path_prefix=training_csv, coder=csv_encoder, header=header)
        )


{'beta1': array([3.6260557], dtype=float32), 'beta2': array([4.240515], dtype=float32), 'hour': array([4], dtype=int64), 'humidity': array([24.640224], dtype=float32), 'weekday': array([6], dtype=int64)}
{'beta1': array([-4.905572], dtype=float32), 'beta2': array([1.1259594], dtype=float32), 'hour': array([8], dtype=int64), 'humidity': array([10.155988], dtype=float32), 'weekday': array([6], dtype=int64)}
{'beta1': array([-3.9921415], dtype=float32), 'beta2': array([-0.56665754], dtype=float32), 'hour': array([5], dtype=int64), 'humidity': array([11.319303], dtype=float32), 'weekday': array([4], dtype=int64)}
{'beta1': array([2.0024812], dtype=float32), 'beta2': array([-2.0756834], dtype=float32), 'hour': array([13], dtype=int64), 'humidity': array([24.520132], dtype=float32), 'weekday': array([5], dtype=int64)}
{'beta1': array([-0.23993467], dtype=float32), 'beta2': array([-3.456368], dtype=float32), 'hour': array([15], dtype=int64), 'humidity': array([20.436716], dtype=float32), 'wee

In [20]:
!echo "Reading from: " $training_csv-00000-of-00001
!cat $training_csv-00000-of-00001 | tail -10

"Reading from: " C:\Users\wgi\AppData\Local\Temp\tmpj9tjlo0n\training.csv-00000-of-00001
beta1,beta2,hour,humidity,weekday
3.6260557,4.240515,4,24.640224,6
-4.905572,1.1259594,8,10.155988,6
-3.9921415,-0.56665754,5,11.319303,4
2.0024812,-2.0756834,13,24.520132,5
-0.23993467,-3.456368,15,20.436716,6


### Serious transformation: Scale $\beta_1$ and $\beta_2$

In [21]:
for phase_csv in [signature_csv_train, signature_csv_eval, signature_csv_test]:
    measure(20000).to_csv(phase_csv, index=None)

In [22]:
def process_data(row):
    for c in ['beta1', 'beta2']:
        row[c] = tft.scale_to_0_1(row[c])
    return row

In [23]:
signature_metadata = dataset_metadata.DatasetMetadata(schema)

In [24]:
csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)    
tfr_encoder = tft.coders.ExampleProtoCoder(schema)            

metadata_dir = os.path.join(temp_dir, "metadata")
with beam.Pipeline('DirectRunner', PipelineOptions()) as p:

    #
    # The context is provided for the AnalyseAndTransform step.
    # That step needs a hand to do its magic.
    #
    with tft_beam.Context(temp_dir=temp_dir):

        #
        # Read from csv, skip headers. Note that we use ordered columns in the encoder
        # 
        signature_data = ( p | 'read_from_csv' 
            >> beam.io.ReadFromText(
                 file_pattern=signature_csv_train, 
                coder=csv_encoder, skip_header_lines=1))

        #
        # attach the metadata: required for AnalyzeAndTransform
        #
        signature_data = ( signature_data, signature_metadata)

        #
        # Do the magic two steps and return also the transform-function
        #
        data_and_metadata, transform_fn = ( signature_data | "AnalyzeAndTransform" 
                         >> beam_impl.AnalyzeAndTransformDataset(process_data))
        
        #
        # split data and metadata
        #
        training_data, training_metadata = data_and_metadata

        #
        # Write the resulting data to a csv file
        #
        _ = (training_data | 'write_to_csv' 
             >> beam.io.WriteToText(
                 file_path_prefix=training_csv, coder=csv_encoder, header=header))

        #
        # For production purposes, we use the TFRecord format
        #
        _ = (training_data | 'write_to_tfr' 
             >> beam.io.WriteToTFRecord(
                 file_path_prefix=training_tfr, coder=tfr_encoder))

        
        #  Process evaluation data with the obtained transform_fn
        #
        signature_data = ( p | 'read_from_csv_eval' 
            >> beam.io.ReadFromText(
                 file_pattern=signature_csv_eval, coder=csv_encoder, skip_header_lines=1))

        signature_data = (signature_data, signature_metadata)

        # Use the transform_fn of the previous step here
        eval_data, _ = ((signature_data, transform_fn) 
                     | "TransformEval" >> tft_beam.TransformDataset())

        _ = (eval_data | 'write_to_tfr_eval' 
             >> beam.io.WriteToTFRecord(
                 file_path_prefix=eval_tfr, coder=tfr_encoder))

        
        
        #
        # Eventually, save the transform function for re-use at prediction time.
        #
        _ = (transform_fn | 'WriteTransformFn' 
             >> transform_fn_io.WriteTransformFn(metadata_dir))


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Assets added to graph.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [30]:
!echo "Reading from: " $training_csv-00000-of-00001
!cat $training_csv-00000-of-00001 | tail -5
!echo metadata is here:
!ls $metadata_dir
!echo TFRecords are here: $training_tfr*

"Reading from: " C:\Users\wgi\AppData\Local\Temp\tmpj9tjlo0n\training.csv-00000-of-00001
0.94582903,0.15036847,10,29.717438,6
0.500492,0.849052,19,25.782593,1
0.8265379,0.7400698,6,24.348576,5
0.17302418,0.9146034,6,6.3833246,0
0.9430103,0.6158343,0,28.630589,6
metadata is here:
transform_fn
transformed_metadata
TFRecords are here: C:\Users\wgi\AppData\Local\Temp\tmpj9tjlo0n\training.tfr*


### Wrap up

We're now able to analyze and process any size of data, scale particular features to the interval $[0, 1]$ while saving the function that actually did it for later. We'll need that function to apply exactly the same scaling to the incoming data at prediction time. 
Note that by simply swapping ```'DirectRunner'``` in the Apache Beam pipeline by ```'DataFlowRunner'``` in an adequately configured GCP environment, we could have the pipeline executed on an arbitrarily large cluster.